# Spatial Signal Separation for Geological magnetic surveys

In [ ]:
import magpylib as magpy
import numpy as np
import matplotlib.pyplot as plt
from magprime.algorithms import PFSS

ImportError: attempted relative import with no known parent package

#### Create Simulation

In [2]:
"10m x 10 m grid"
m1 = magpy.misc.Dipole(moment=(0.522, -0.904, 0.930), position=(0, 0, 0))
m2 = magpy.misc.Dipole(moment=(0.222, 0.504, -0.430), position=(1, -2, 0))
m3 = magpy.misc.Dipole(moment=(0.522, -0.904, 0.930), position=(4, 1, 0))
m4 = magpy.misc.Dipole(moment=(0.222, 0.504, -0.430), position=(-3, -2, 0))
m5 = magpy.misc.Dipole(moment=(0.522, -0.904, 0.930), position=(2, 4, 0))
m6 = magpy.misc.Dipole(moment=(0.222, 0.504, -0.430), position=(-4, 0, 0))
m7 = magpy.misc.Dipole(moment=(0.522, -0.904, 0.930), position=(1, 1, 0))
m8 = magpy.misc.Dipole(moment=(0.222, 0.504, -0.430), position=(2, -2, 0))
background = magpy.misc.Dipole(moment=(0,1000,10000), position=(10, 10, 0))

collection = magpy.Collection(m1, m2, m3, m4, m5, m6, m7, m8, background)
ts = np.linspace(-5, 5, 50)
grid = np.array([[(x, y, .5) for x in ts] for y in ts])
B = collection.getB(grid)
B = np.moveaxis(B, -1, 0)
Bmag = np.linalg.norm(B, axis=0)   

#### Separate Regional Anomalies from Background Field

In [3]:
Xd, Xs = PFSS.pfss(Bmag, 5, beta=0.5, max_iter=100, eps=1e-4, verbose=False)

NameError: name 'PFSS' is not defined

In [ ]:
fig, ax = plt.subplots(1,3, figsize = (10, 4), sharex=True)
ax[0].imshow(np.log(Bmag+1e-3), extent=[-5, 5, -5, 5], origin='lower')
ax[0].set_title("Mixed Field")

# Overlay dipole locations
positions = np.array([m.position[:2] for m in (m1, m2, m3, m4, m5, m6, m7, m8)])
ax[0].scatter(positions[:,0], positions[:,1], marker='x')

ax[1].imshow(np.log(Xd+1e-3), extent=[-5, 5, -5, 5], origin='lower')
ax[1].set_title("Regional Field")

ax[2].imshow(np.log(Xs+1e-3), extent=[-5, 5, -5, 5], origin='lower')
ax[2].set_title("Anomaly Field")

ax[0].set_xlabel('x (m)')
ax[2].set_xlabel('x (m)')
ax[1].set_xlabel('x (m)')
ax[0].set_ylabel('y (m)')

# Singular Spectrum Analysis

In [ ]:
comps = PFSS.ssa(Bmag, 10)

In [ ]:
fig, ax = plt.subplots(1,3, figsize = (10, 4), sharex=True)
ax[0].imshow(np.log(comps[0]+1e-3), extent=[-5, 5, -5, 5], origin='lower')
ax[0].set_title("Comp 0")

ax[1].imshow(np.log(comps[1]+1e-3), extent=[-5, 5, -5, 5], origin='lower')
ax[1].set_title("Comp 1")

ax[2].imshow(np.log(comps[2]), extent=[-5, 5, -5, 5], origin='lower')
ax[2].set_title("Comp 2")
ax[0].set_xlabel('x (m)')
ax[2].set_xlabel('x (m)')
ax[1].set_xlabel('x (m)')
ax[0].set_ylabel('y (m)')